# Instructions

This colab contains code used to generate the Figures 3, 4, and 6 in the paper "Personality Traits in Large Language Models" (https://arxiv.org/pdf/2307.00184). The code below assumes that all the data consumed in the colab (especially the pickled dataframes) lives in a local filesystem either in a cloud instance running a Jupyter notebook such as Google Colab or a desktop. But those file I/O operations can easily be replaced to use any other file management solutions. The inline comments for some of the operations explain the motivation behind them and what to expect in the results of running an analysis in a cell.

To run this colab:
1. Connect to an appropriate runtime. (For instance, if running the bulk inference directly from the colab, connect to a GPU kernel.)
2. Check experiment parameters below.
3. Run the code cells for analysis.

In [ ]:
%pip install plotly
%pip install pandas
%pip install scipy
%pip install numpy
%pip install pickle

In [ ]:
import plotly.express as px

import pandas as pd

In [ ]:
#@markdown Locations of the dataframe that are the source for the figures
CONVERGENT_VALIDITY_DF = 'figures_data/convergent_validity.pkl'
EXTERNAL_VALIDITY_AGR_DF = 'figures_data/external_validity_AGR.pkl'
EXTERNAL_VALIDITY_EXT_DF = 'figures_data/external_validity_EXT.pkl'
EXTERNAL_VALIDITY_CON_DF = 'figures_data/external_validity_CON.pkl'
EXTERNAL_VALIDITY_NEU_DF = 'figures_data/external_validity_NEU.pkl'
EXTERNAL_VALIDITY_OPE_DF = 'figures_data/external_validity_OPE.pkl'
MODEL_NAME_MAPPING = {
    'palm_62b_q': 'PaLM 62B',
    'flan_palm_8b_q': 'Flan-PaLM 8B',
    'flan_palm_62b_q': 'Flan-PaLM 62B',
    'flan_palm_540b_q': 'Flan-PaLM 540B',
    'flan_palmchilla_62b_q': 'Flan-PaLMChilla 62B',
}

## Fig 3: Convergent Correlations between IPIP-NEO and BFI scores by mode

In [ ]:
raw_df = pd.read_pickle(CONVERGENT_VALIDITY_DF)

plot_df = []
for col in 'EXT	AGR	CON	NEU	OPE Avg'.split():
  work_df = raw_df[['model_id', col]].rename({col: 'Correlation', 'model_id': 'Model'}, axis=1)
  work_df['Measure'] = col
  plot_df.append(work_df)

plot_df = pd.concat(plot_df, axis=0, ignore_index=True)
plot_df['Model'] = plot_df['Model'].map(MODEL_NAME_MAPPING)

fig = px.histogram(plot_df, y='Correlation', color='Measure', x='Model', barmode='group',
                   title='Convergent Correlations Between IPIP-NEO and BFI Scores, by Model',
                   text_auto=True)
fig.data[-1].update(
    name='All correlations (averaged)',
    error_y=dict(
        type='data',
        symmetric=True,
        array=raw_df.set_index('model_id').astype(float).drop('Avg', axis=1).std(axis=1),
    ))
fig.update_layout(title_x=0.5)
fig.update_yaxes(range=[-1, 1],
                 title='Correlation')
fig.update_layout(legend=dict(
    orientation='h',
    yanchor='bottom',
    y=1.02,
    xanchor='right',
    x=1
))
fig

## Fig 4: Criterion validity evidence. IPIP-NEO correlation with all traits

#### Extraversion

In [ ]:
raw_df = pd.read_pickle(EXTERNAL_VALIDITY_EXT_DF)

plot_df = []
for col in 'Watson & Clark, 1992', 'palm_62b_q', 'flan_palm_8b_q', 'flan_palm_62b_q', 'flan_palmchilla_62b_q', 'flan_palm_540b_q':
  work_df = raw_df[['Scale', col]].rename({col: 'Value'}, axis=1)
  work_df['Model'] = col
  plot_df.append(work_df)

plot_df = pd.concat(plot_df, axis=0, ignore_index=True)
plot_df = plot_df[plot_df['Scale'].str.strip().astype(bool)]
plot_df['Model'] = plot_df['Model'].map(MODEL_NAME_MAPPING | {'Watson & Clark, 1992': 'Watson & Clark, 1992'})

fig = px.histogram(plot_df, x='Model', color='Scale', y='Value', barmode='group',
                   title='Correlations of IPIP-NEO Extraversion with Positive and Negative Affect, by model',
                   text_auto=True)

fig.update_layout(title_x=0.5)
fig.update_yaxes(range=[-1, 1],
                 title='Correlation')
fig.update_layout(legend=dict(
    orientation='h',
    yanchor='bottom',
    y=1.02,
    xanchor='right',
    x=1
))
fig.for_each_annotation(lambda a: a.update(font=dict(size=22)))
fig.update_layout(font=dict(size=22), title=dict(text=''))
fig.update_xaxes(tickangle=15)
fig

#### Aggreableness

In [ ]:
raw_df = pd.read_pickle(EXTERNAL_VALIDITY_AGR_DF)

plot_df = []
for col in 'palm_62b_q', 'flan_palm_8b_q', 'flan_palm_62b_q', 'flan_palmchilla_62b_q', 'flan_palm_540b_q':
  work_df = raw_df[['Scale', col]].rename({col: 'Value'}, axis=1)
  work_df['Model'] = col
  plot_df.append(work_df)

plot_df = pd.concat(plot_df, axis=0, ignore_index=True)
plot_df = plot_df[plot_df['Scale'].str.strip().astype(bool)]
plot_df['Model'] = plot_df['Model'].map(MODEL_NAME_MAPPING | {})

fig = px.histogram(plot_df, x='Model', y='Value', color='Scale', barmode='group',
                   title='Correlations of IPIP Agreeableness with BPAQ aggression scales, by model',
                   text_auto=True)
fig.update_layout(title_x=0.5)
fig.update_yaxes(range=[-1, 1],
                 title='Correlation')
fig.update_layout(legend=dict(
    orientation='h',
    yanchor='bottom',
    y=1.02,
    xanchor='right',
    x=1
))
fig.for_each_annotation(lambda a: a.update(font=dict(size=22)))
fig.update_layout(font=dict(size=22), title=dict(text=''))
fig.update_xaxes(tickangle=15)
fig

#### Conscientiousness

In [ ]:
raw_df = pd.read_pickle(EXTERNAL_VALIDITY_CON_DF)

plot_df = []
for col in 'palm_62b_q', 'flan_palm_8b_q', 'flan_palm_62b_q', 'flan_palmchilla_62b_q', 'flan_palm_540b_q':
  work_df = raw_df[['Scale', col]].rename({col: 'Value'}, axis=1)
  work_df['Model'] = col
  plot_df.append(work_df)

plot_df = pd.concat(plot_df, axis=0, ignore_index=True)
plot_df = plot_df[plot_df['Scale'].str.strip().astype(bool)]
plot_df['Model'] = plot_df['Model'].map(MODEL_NAME_MAPPING)

fig = px.histogram(plot_df, x='Model', y='Value', color='Scale', barmode='group',
                   title='Correlations of IPIP Conscientiousness with related human values, by model',
                   text_auto=True)
fig.update_layout(title_x=0.5)
fig.update_yaxes(range=[-1, 1],
                 title='Correlation')
fig.update_layout(legend=dict(
    orientation='h',
    yanchor='bottom',
    y=1.02,
    xanchor='right',
    x=1
))
fig.for_each_annotation(lambda a: a.update(font=dict(size=22)))
fig.update_layout(font=dict(size=22), title=dict(text=''))
fig.update_xaxes(tickangle=15)
fig

#### Neuroticism

In [ ]:
raw_df = pd.read_pickle(EXTERNAL_VALIDITY_NEU_DF)

plot_df = []
for col in 'Watson & Clark, 1992', 'palm_62b_q', 'flan_palm_8b_q', 'flan_palm_62b_q', 'flan_palmchilla_62b_q', 'flan_palm_540b_q':
  work_df = raw_df[['Scale', col]].rename({col: 'Value'}, axis=1)
  work_df['Model'] = col
  plot_df.append(work_df)

plot_df = pd.concat(plot_df, axis=0, ignore_index=True)
plot_df = plot_df[plot_df['Scale'].str.strip().astype(bool)]
plot_df['Model'] = plot_df['Model'].map(MODEL_NAME_MAPPING | {'Watson & Clark, 1992': 'Watson & Clark, 1992'})

fig = px.histogram(plot_df, x='Model', y='Value', color='Scale', barmode='group',
                   title='Correlations of IPIP Neuroticism with PA and NA, by model',
                   text_auto=True)
fig.update_layout(title_x=0.5)
fig.update_yaxes(range=[-1, 1],
                 title='Correlation')
fig.update_layout(legend=dict(
    orientation='h',
    yanchor='bottom',
    y=1.02,
    xanchor='right',
    x=1
))
fig.for_each_annotation(lambda a: a.update(font=dict(size=22)))
fig.update_layout(font=dict(size=22), title=dict(text=''))
fig.update_xaxes(tickangle=25)
fig

#### Openness

In [ ]:
raw_df = pd.read_pickle(EXTERNAL_VALIDITY_OPE_DF)

plot_df = []
for col in 'palm_62b_q', 'flan_palm_8b_q', 'flan_palm_62b_q', 'flan_palmchilla_62b_q', 'flan_palm_540b_q':
  work_df = raw_df[['Scale', col]].rename({col: 'Value'}, axis=1)
  work_df['Model'] = col
  plot_df.append(work_df)

plot_df = pd.concat(plot_df, axis=0, ignore_index=True)
plot_df = plot_df[plot_df['Scale'].str.strip().astype(bool)]
plot_df['Model'] = plot_df['Model'].map(MODEL_NAME_MAPPING)

fig = px.histogram(plot_df, x='Model', y='Value', color='Scale', barmode='group',
                   title='Correlations of IPIP Openness with creativity scales, by model',
                   text_auto=True)

fig.update_layout(title_x=0.5)
fig.update_yaxes(range=[-1, 1],
                 title='Correlation')
fig.update_layout(legend=dict(
    orientation='h',
    yanchor='bottom',
    y=1.02,
    xanchor='right',
    x=1
))
fig.for_each_annotation(lambda a: a.update(font=dict(size=22)))
fig.update_layout(font=dict(size=22), title=dict(text=''))
fig.update_xaxes(tickangle=15)
fig

## Fig 6b: IPIP-NEO Relevance to Generated Text

In [ ]:
raw_data_llm = """[{"index":0,"Variable1":"IPIP300-EXT","Variable2":"ams-IPIP300-EXT","pearson":0.5778892465339398,"pearson_p":1.2052054711338261e-200,"spearman":0.5779114129466953,"spearman_p":1.1541629435322971e-200},{"index":1,"Variable1":"IPIP300-AGR","Variable2":"ams-IPIP300-AGR","pearson":0.6741851211419059,"pearson_p":3.399372260086691e-298,"spearman":0.6122784687197989,"spearman_p":1.252372624796226e-231},{"index":2,"Variable1":"IPIP300-CON","Variable2":"ams-IPIP300-CON","pearson":0.5395665544984065,"pearson_p":3.379961176444279e-170,"spearman":0.6162234916929736,"spearman_p":1.9753235475172855e-235},{"index":3,"Variable1":"IPIP300-NEU","Variable2":"ams-IPIP300-NEU","pearson":0.5886821617265545,"pearson_p":5.713724053146592e-210,"spearman":0.6137409703770154,"spearman_p":4.9488728930903925e-233},{"index":4,"Variable1":"IPIP300-OPE","Variable2":"ams-IPIP300-OPE","pearson":0.3780319835499575,"pearson_p":2.344533227977254e-77,"spearman":0.44489890013553535,"spearman_p":8.100016932050919e-110}]"""

plot_df = pd.read_json(raw_data_llm)
plot_df['Measure'] = plot_df['Variable1'].apply(lambda s: s.split('-')[1])
plot_df = plot_df[['Measure', 'pearson']]
plot_df['Respondent'] = 'Flan-PaLM 540B'

plot_df = pd.concat([plot_df, pd.DataFrame({'Respondent': ['Human', 'Human', 'Human', 'Human', 'Human'],
                                            'Measure': ['EXT', 'AGR', 'CON', 'NEU', 'OPE'], 'pearson': [0.42, 0.35, 0.37, 0.35, 0.43]})], ignore_index=True)
plot_df = plot_df.round(2)


fig = px.histogram(
    plot_df, x='Measure', y='pearson', color='Respondent', pattern_shape='Respondent', barmode='group',
    title='Accuracy of IPIP-NEO Scores in Predicting Personality Observed in Social Media Status Updates',
    text_auto=True)

fig.update_layout(title_x=0.5)
fig.update_yaxes(
    range=[0, 1],
    title="Pearson's <i>r</i>")
fig.update_layout(legend=dict(
    orientation='h',
    yanchor='bottom',
    y=1.02,
    xanchor='right',
    x=1
))
fig.for_each_annotation(lambda a: a.update(font=dict(size=22)))
fig.update_layout(font=dict(size=22), title=dict(text=''))
fig.update_xaxes(tickangle=15)
fig